### torch.nn.Module 그리고 torch.nn Parameter

* Parameter를 제외하고 torch.nn.Module의 하위 클래스.
* 이는 PyTorch 모델 및 해당 구성 요소와 관련된 동작을 캡슐화하기 위한 PyTorch 기본 클래스임.

* torch.nn.Module의 중요한 동작 중 하나는 매개변수를 등록하는 것.
* 특정 Module 하위 클래스가 학습 가중치를 가지고 있는 경우, 이러한 가중치는 torch.nn.Parameter의 인스턴스로 표현됨.
* Parameter 클래스는 torch.Tensor의 하위 클래스로, 특별한 동작을 가지고 있음.
* Module의 속성으로 할당될 때 해당 모듈의 매개변수 목록에 추가됨.
* 이러한 매개변수는 Module 클래스의 parameters() 메서드를 통해 액세스할 수 있음.


* 간단한 예로, 두 개의 선형 레이어와 활성화 함수가 있는 매우 간단한 모델이 있음.
* 이 모델의 인스턴스를 만들고 그 매개변수에 대해 보고를 요청할 것.

In [ ]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()

print('The model:')
print(tinymodel)

print('\n\nJust one layer:')
print(tinymodel.linear2)

print('\n\nModel params:')
for param in tinymodel.parameters():
    print(param)

print('\n\nLayer params:')
for param in tinymodel.linear2.parameters():
    print(param)

The model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[-0.0761,  0.0504, -0.0850,  ...,  0.0445,  0.0746,  0.0464],
        [ 0.0094, -0.0644,  0.0416,  ..., -0.0369, -0.0182, -0.0972],
        [-0.0099,  0.0432,  0.0248,  ...,  0.0085, -0.0284, -0.0254],
        ...,
        [ 0.0192,  0.0655, -0.0498,  ...,  0.0789,  0.0757,  0.0232],
        [-0.0903, -0.0237,  0.0134,  ..., -0.0915,  0.0524, -0.0818],
        [ 0.0074, -0.0654,  0.0797,  ...,  0.0417,  0.0788,  0.0180]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0861,  0.0838,  0.0176,  0.0172,  0.0621,  0.0358, -0.0537, -0.0049,
         0.0286,  0.0464,  0.0868, -0.0407, -0.0046,  0.0711,  0.0307, -0.0494,
         0.0094,  0.0830,  0.0481,  0.03

* 이것은 PyTorch 모델의 기본 구조를 보여줌.
* 모델의 레이어 및 다른 구성 요소를 정의하는 init() 메서드가 있고, 계산이 수행되는 forward() 메서드가 있음.
* 모델 또는 그 하위 모듈 중 하나를 출력하여 구조에 대해 알아볼 수 있음.

### Common Layer Types
### Linear Layers
* 가장 기본적인 유형의 신경망 레이어는 선형 또는 완전히 연결된 레이어임.
* 이는 레이어의 가중치에 의해 지정된 정도로 각 입력이 레이어의 모든 출력에 영향을 미치는 레이어임.
* 모델이 m 개의 입력과 n 개의 출력을 가지는 경우, 가중치는 m x n 행렬이 됨.


In [ ]:
lin = torch.nn.Linear(3, 2)
x = torch.rand(1, 3)
print('Input:')
print(x)

print('\n\nWeight and Bias parameters:')
for param in lin.parameters():
  print(param)

y = lin(x)
print('\n\nOutput:')
print(y)

Input:
tensor([[0.4444, 0.7102, 0.5446]])


Weight and Bias parameters:
Parameter containing:
tensor([[-0.5363,  0.4116,  0.5762],
        [ 0.0862, -0.3749, -0.1343]], requires_grad=True)
Parameter containing:
tensor([ 0.1639, -0.3172], requires_grad=True)


Output:
tensor([[ 0.5317, -0.6183]], grad_fn=<AddmmBackward0>)


* 만약 선형 레이어의 가중치로 x의 행렬 곱셈을 수행하고 편향을 더한다면, 출력 벡터 y를 얻게 될 것.
* 다른 중요한 기능 하나를 더 보자면, 우리가 lin.weight로 레이어의 가중치를 확인했을 때, 그것은 자신을 Paramenter(텐서의 하위 클래스)로 보고하고 autograd로 그래디언트를 추적하고 있을음 알림.
* 이것은 tensor와는 달리 Parameter의 기본 동작임.

* 선형 레이어는 딥러닝 모델에서 널리 사용됨.
* 가장 흔하게 볼 수 있는 곳 중 하나는 분류기 모델임.
* 분류기 모델에서는 보통 끝 부분에 하나 이상의 선형 레이어가 있으며, 마지막 레이어는 분류기 처리하는 클래스의 수인 n개의 출력을 가짐.

### Convolution Layer
* 합성곱 레이어는 공간적 상관 관계가 높은 데이터를 처리하기 위해 만들어짐.
* 컴퓨터 비전에서 매우 일반적으로 사용되며, 여기에서 특성의 밀집된 그룹을 감지하고 더 높은 수준의 특성으로 구성함.
* 이러한 레이어는 다른 맥락에서도 나타남.
* 예를 들어, NLP(Natural Language Processing) 응용 프로그램에서는 단어의 즉각적인 컨텍스트(즉, 시퀀스 내에서 근처에 있는 다른 단어)가 문장의 의미에 영향을 미칠 수 있음.
* 이전 비디오에서 LeNet5에서 합성곱 레이어가 작동하는 것을 확인함.

In [ ]:
import torch.functional as F

class LeNet(torch.nn.Module):

  def __init__(self):
    super(LeNet, self).__init__()
    # 1 input image channel (black and white), 6 output channels, 5 x 5 square convolution
    # kernel
    self.conv1 = torch.nn.Conv2d(1, 6, 5)
    self.conv2 = torch.nn.Conv2d(6, 16, 3)
    # an affine operation: y = Wx + b
    self.fc1 = torch.nn.Linear(16 * 6 * 6, 120) # 6 x 6 from image dimension
    self.fc2 = torch.nn.Linear(120, 84)
    self.fc3 = torch.nn.Linear(84, 10)

  def forward(self, x):
    # Max pooling over a (2, 2) window
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    # If the size is a square you can only specify a single number
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:] # all dimensions except the batch dimension
    num_features = 1
    for s in size:
      num_features *= s
    return num_features

* 이 모델의 컨볼루션 레이어에서 일어나는 과정을 살펴보겠음.

* 우선 conv1 부터 시작하겠음.
  + LeNet5 는 1 x 32 x 32 흑백 이미지를 입력으로 받음.
  +  컨볼루션 레이어의 생성자에 대한 **첫 번째 인자는 입력 채널의 수**.
  + 여기서는 1임.
  + 만약 3개의 컬러 채널을 다루도록 구성한다면 이 값은 3이 될 것.
  + 컨볼루션 레이어는 이미지를 스캔하면서 인식하는 패턴을 찾는 창처럼 작동함.
  + 이러한 패턴을 특징이라고 하며, 컨볼루션 레이어의 두 번째 인자는 학습할 특징의 수임.
  + 여기서는 레이어가 6개의 특징을 학습하도록 설정함.
  + 위에서 컨볼루션 레이어를 창에 비유했지만, 이 창의 크기는 얼마일까요?
  + 세 번째 인자는 창 또는 커널 크기임.
  + 여기서 "5"는 5 x 5 커널을 선택했다는 것을 의미함. (만약 높이와 너비가 다른 커널을 원한다면 이 인자에 튜플을 지정할 수 있음.)


* 컨볼루션 레이어의 출력은 활성화 맵.
* 입력 텐서에서 특징의 존재를 나타내는 공간적 표현.
* conv1은 출력 텐서를 6x28x28로 제공할 것.
* 여기서 6은 특징의 수, 28은 맵의 높이와 너비. (28은 32픽셀 행을 스캔할 때 5픽셀 창으로 스캔하는 경우 유효한 위치가 28개만 있기 때문에 나옴.)

* 다음 컨볼루션 레이어인 conv2는 6개의 입력 채널을 기대하며 (첫 번째 레이어에서 찾은 6개의 특징에 해당.), 16개의 출력 채널과 3 x 3 커널을 가짐.
* 이는 16 x 12 x 12 활성화 맵을 출력하며, 다시 한 번 최대 풀링 레이어에 의해 16 x 6 x 6 로 줄어듦.
* 이 출력을 선형 레이어에서 소비할 수 있도록 16 x 6 x 6 = 576 요소로 재구성됨.

* 컨볼루션 레이어는 1D, 2D 및 3D 텐서를 다루기 위한 것.
* 또한 입력의 스트라이드 길이(예 : 매 2 번째 또는 매 3 번째 위치만 스캔), 패딩(입력의 가장자리까지 스캔할 수 있또록)과 같은 많은 추가 인자가 있음.

### 순환 레이어
* 순환 신경망 (또는 RNN) 은 순차적 데이터에 사용됨.
* 과학 기기의 시계열 측정값부터 자연어 문장, DNA 염기까지 다양한 데이터 형태를 다룸.
* RNN은 현재까지 본 시퀀스에 대한 일종의 메모리 역할을 하는 숨겨진 상태를 유지함으로써 이를 수행함.
* 순환 신경망 레이어의 내부 구조 또는 그 변형인 LSTM (장단기 기억) 및 GRU (게이트 순환 유닛)은 다소 복잡하며 이 비디오의 범위를 벗어나지만, 우리는 LSTM 기반의 품사 태거(단어가 명사, 동사 등인지를 알려주는 분류기의 한 유형)를 통해 어떻게 작동하는지 보여줌.

In [ ]:
class LSTMTagger(torch.nn.Module):

  def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
    super(LSTMTagger, self).__init__()
    self.hidden_dim = hidden_dim

    self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

    # The LSTM takes word embeddings as inputs, and outputs hidden states
    # With dimensionality hidden_dim
    self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

    # The linear layer that maps from hidden state space to tag space
    self.hidden2tag = torch.nn.Linear(hidden_dim, target_size)

  def forward(self, sentence):
    embeds = self.word_embeddings(sentence)
    lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
    target_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
    target_scores = F.log_softmax(target_space, dim=1)
    return target_scores

* 생성자에는 네 가지 인수가 있음.
  + vocab_size 는 입력 어휘의 단어 수.
  + 각 단어는 vocab_size 차원 공간에서의 원핫 벡터(또는 단위 벡터)
  + target_size는 출력 집합의 태그 수.
  + embedding_dim은 어휘 인베딩 공간의 크기.
  + 임베딩은 어휘를 저차원 공간에 매핑하는데, 이 공간에서 의미한 유사한 단어들은 공간 상에서 서로 가까이 위치함.
  + hidden_dim 은 LSTM의 메모리 크기.

* 입력은 단어가 원핫 벡터의 인덱스로 표현된 문장임.
* 임베딩 레이어는 이를 embedding_dim 차원의 공간으로 매핑함.
* LSTM은 이러한 임베딩 시퀀스를 가져와 이를 반복하여 hidden_dim 길이의 출력 벡터를 생성함.
* 최종 선형 레이어는 분류기 역할을 하며, 최종 레이어의 출력에 log_softmax()를 적용하여 주어진 단어가 주어진 태그로 매핑될 확률을 정규화된 추정 확률 집합으로 변환함.

### Transformer
* 트랜스포머는 BERT와 같은 모델을 통해 자연어 처리의 최첨단 기술로 등장하여 다양한 용도로 사용되는 네트워크임.
* PyTorch에는 트랜스포머 모델의 전반적인 매개변수를 정의할 수 있는 Transformer 클래스가 있음.
* 이 클래스는 어텐션 헤드의 수, 인코더 및 디코더 레이어의 수, 드롭아웃 및 활성화 함수 등을 설정할 수 있음. (적절한 매개변수를 사용하여 이 클래스에서 BERT 모델을 빌드할 수 있음.)
* torch.nn.Transformer 클래스에는 개별 구성 요소 (TransformerEncoder, TransformerDecoder)와 하위 구성 요소 (TransformerEncoderLayer, TransformerDecoderLayer)를 캡슐화하는 클래스도 있음.

### Other Layers and Functions
### Data Manipulation Layers
* 최대 풀링 (그리고 그와 쌍을 이루는 최소 풀링)은 텐서를 셀을 결합하고, 입력 셀의 최댓값을 출력 셀에 할당하여 텐서를 줄임.

In [ ]:
my_tensor = torch.rand(1, 6, 6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor))

tensor([[[0.1669, 0.6670, 0.0691, 0.9314, 0.9236, 0.7897],
         [0.4204, 0.4785, 0.7635, 0.7198, 0.7221, 0.9211],
         [0.1150, 0.0205, 0.4211, 0.8410, 0.1107, 0.9597],
         [0.1470, 0.0499, 0.3237, 0.0457, 0.6914, 0.3067],
         [0.2798, 0.2738, 0.2144, 0.1522, 0.9379, 0.7042],
         [0.5752, 0.1813, 0.7238, 0.3005, 0.4858, 0.9441]]])
tensor([[[0.7635, 0.9597],
         [0.7238, 0.9441]]])


* 위의 값을 자세히 살펴보면 maxpooled 출력의 각 값이 6 x 6 입력의 각 사분면의 최대값임을 알 수 있음.

* 정규화 레이어는 한 레이어의 출력을 다른 레이어에 공급하기 전에 다시 중앙에 배치하고 정규화함.
* 중간 텐서를 중앙에 배치하고 크기를 조정하면 경사도가 폭발하거나 사라지지 않고 더 높은 학습률을 사용할 수 있는 등 여러 가지 유익한 효과가 있음.

In [ ]:
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)

print(my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)

print(normed_tensor.mean())

tensor([[[15.7795,  7.6450, 22.3092, 23.8927],
         [18.5447, 23.5124,  6.5881,  9.8195],
         [19.4506, 13.2722, 16.2240, 17.1098],
         [ 7.9119, 14.2600, 18.5712,  5.3012]]])
tensor(15.0120)
tensor([[[-0.2541, -1.5243,  0.7656,  1.0128],
         [ 0.5823,  1.3187, -1.1900, -0.7110],
         [ 1.3275, -1.4656, -0.1312,  0.2693],
         [-0.6897,  0.5268,  1.3530, -1.1900]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(-1.0431e-07, grad_fn=<MeanBackward0>)


* 위의 셀을 실행하여 입력 텐서에 큰 배율 인수와 오프셋을 추가함.
* 입력 텐서는 15 근처 어딘가에 표시되어야 함.
* 정규화 레이어를 통해 실행한 후에는 값이 더 작고 0을 중심으로 그룹화되어 있음을 알 수 있음.
* 실제로 평균은 매우 작아야 함. ( > 1e-8)

* 이는 많은 활성화 함수가 0 근처에서 가장 강한 기울기를 갖기 때문에 유용하지만 때로는 입력에 대해 기울기가 사라지거나 폭발하여 0에서 멀리 떨어져 있기 때문에 유용함.
* 가장 가파른 경사 영역을 중심으로 데이터를 유지하면 더 빠르고 더 나은 학습과 더 높은 실행 가능한 학습 속도를 의미하는 경향이 있음.

* 드롭아웃 레이어는 모델에서 희소 표현을 장려하는 도구.
* 즉, 더 적은 데이터로 추론을 수행하도록 유도하는 도구.

* 드롭아웃 레이어는 학습 중에 입력 텐서의 일부를 무작위로 설정하여 작동함.
* 드롭아웃 레이어는 추론을 위해 항상 꺼져 있음.
* 이렇게 하면 모델이 마스크되거나 축소된 데이터 세트에 대해 학습하게 됨.

In [ ]:
my_tensor = torch.rand(1, 4, 4)

dropout = torch.nn.Dropout(p=0.4)
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[0.0000, 0.3696, 0.0000, 0.0000],
         [1.5895, 0.9548, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.2475, 0.6377],
         [0.0000, 0.2069, 0.0000, 1.5246]]])
tensor([[[0.6939, 0.0000, 0.0000, 0.0000],
         [1.5895, 0.9548, 0.5724, 1.3018],
         [0.4830, 0.0000, 0.0000, 0.0000],
         [1.3501, 0.2069, 0.0000, 0.0000]]])


* 위에서 샘플 텐서에 대한 드롭아웃 효과를 볼 수 있음.
* 선택적 p 인수를 사용하여 개별 가중치가 떨어질 확률을 설정할 수 있음.
* 그렇지 않으면 기본값은 0.5임.

### Activation Function
* 활성화 기능을 통해 딥러닝이 가능해짐.
* 신경망은 실제로 수학 함수를 시뮬레이션하는 (많은 매개변수가 있는) 프로그램임.
* 우리가 한 모든 것이 레이어 가중치별로 여러 텐서를 반복적으로 수행하는 것뿐이었다면 선형 함수만 시뮬레이션할 수 있었음.
* 또한, 전체 네트워크가 단일 행렬 곱셈으로 축소될 수 있으므로 많은 레이어를 갖는 것은 의미가 없음.
* 레이어 사이에 비선형 함수를 삽입하면 딥러닝 모델이 선형 함수가 아닌 모든 함수를 시뮬레이션할 수 있음.

* torch.nn.Module 은 ReLU 및 그 변형인 Tanh, Hardtanh, sigmoid 등을 포함한 모든 주요 활성화 함수를 캡슐화하는 객체가 있음.
* 또한 모델의 출력 단계에서 가장 유용한 Softmax 와 같은 다른 기능도 포함되어 있음.

### Loss function
* 손실 함수는 모델의 예측이 정답에서 얼마나 멀리 떨어져 있는지 알려줌.
* PyTorch에는 일반적인 MSE(평균 제곱 오류 = L2 Norm), 교차 엔트로피 손실 및 음의 우도 손실(classifier에 유용함) 등을 포함한 다양한 손실 함수가 포함되어 있음.